# Intro

## Assignment Machine Learning

| Nama          | NIM |
|---------------|-----|
| Muhammad Iqbal Saputra | 2702390236 |
| Radit Arya Pradipta               | 2802446705          |
| Muhammad Reza Alghifari                   | 2802555044          |

# Import Library

# Exploratory Data Analysis

# Pre-processing

# Feature Engineering

# Model Training and Evaluation